In [22]:
# # Read from:
# Dune - OP Deployed by deployer address type 
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [25]:
import pandas as pd
import numpy as np

In [26]:
tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
distrib_df = pd.read_csv('csv_outputs/dune_op_distribution_type.csv')
program_df = pd.read_csv('inputs/OP Summer Programs [Public].csv')

In [33]:
#Filter TVL DF
tvl = tvl[tvl['include_in_summary'] == 1]
tvl['join_key'] = tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')
display(tvl)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_23367/3549945408.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tvl['join_key'] = tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')


,date,include_in_summary,top_level_name,program_name,app_name,num_op,num_op_override,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,flows_retention,last_price_net_dollar_flows_retention,join_key
0,2023-03-10,1,OP/agEUR: Velodrome,Angle - OP/agEUR: Velodrome,Angle,NaN,NaN,During Program,Gov Fund - Season 1,2023-02-20,NaN,5.635648e+05,5.635648e+05,5.572557e+05,5.572557e+05,NaN,NaN,Angle - OP/agEUR: Velodrome
1,2023-03-10,1,agEUR/MAI: Velodrome,Angle - agEUR/MAI: Velodrome,Angle,NaN,NaN,During Program,Gov Fund - Season 1,2023-02-20,NaN,1.035245e+06,1.035245e+06,1.037579e+06,1.037579e+06,NaN,NaN,Angle - agEUR/MAI: Velodrome
2,2023-03-10,1,USDC/agEUR: Uniswap,Angle - USDC/agEUR: Uniswap,Angle,NaN,NaN,During Program,Gov Fund - Season 1,2023-02-20,NaN,7.990376e+05,7.990376e+05,7.995267e+05,7.995267e+05,NaN,NaN,Angle - USDC/agEUR: Uniswap
3,2023-03-10,1,vWETH/rETH: Velodrome,Rocket Pool - vWETH/rETH: Velodrome,Rocket Pool,NaN,NaN,During Program,Gov Fund - Season 1,2023-02-08,NaN,1.805381e+07,1.805381e+07,1.824065e+07,1.824065e+07,NaN,NaN,Rocket Pool - vWETH/rETH: Velodrome
4,2023-03-10,1,DHT/OP: Velodrome,Dhedge - DHT/OP: Velodrome,Dhedge,NaN,NaN,During Program,Gov Fund - Season 1,2023-02-03,NaN,9.502888e+05,9.502888e+05,9.667531e+05,9.667531e+05,NaN,NaN,Dhedge - DHT/OP: Velodrome
5,2023-03-10,1,USD+/DOLA: Velodrome,Overnight - USD+/DOLA: Velodrome,Overnight,NaN,NaN,During Program,Gov Fund - Season 1,2023-01-30,NaN,4.062132e+06,4.062132e+06,4.123492e+06,4.123492e+06,NaN,NaN,Overnight - USD+/DOLA: Velodrome
6,2023-03-10,1,USD+/LUSD: Velodrome,Overnight - USD+/LUSD: Velodrome,Overnight,NaN,NaN,During Program,Gov Fund - Season 1,2023-01-30,NaN,8.568366e+05,8.568366e+05,9.072528e+05,9.072528e+05,NaN,NaN,Overnight - USD+/LUSD: Velodrome
7,2023-03-10,1,Uniswap LM - Phase 2*,Defiedge - Uniswap LM - Phase 2*,Defiedge,2.500000e+04,2.500000e+04,Post-Program,Gov Fund - Phase 0,2023-01-16,2023-02-08,7.740442e+06,2.130290e+06,7.740442e+06,2.130290e+06,0.275215,0.275215,Defiedge - Uniswap LM - Phase 2
8,2023-03-10,1,Uniswap LM - Phase 2,Xtoken - Uniswap LM - Phase 2,Xtoken,2.500000e+04,2.500000e+04,Post-Program,Gov Fund - Phase 0,2023-01-16,2023-02-08,2.823585e+06,-3.557705e+04,3.027949e+06,7.479739e+04,0.012600,0.024702,Xtoken - Uniswap LM - Phase 2
10,2023-03-10,1,Uniswap LM - Phase 2,Gamma - Uniswap LM - Phase 2,Gamma,2.500000e+04,2.500000e+04,Post-Program,Gov Fund - Phase 0,2023-01-16,2023-02-08,6.290259e+06,-1.238923e+05,6.546800e+06,-1.617516e+04,0.019696,0.002471,Gamma - Uniswap LM - Phase 2


In [31]:
#Set up Distributions for Mapping
distrib_df['program_map'] = np.where(
                                distrib_df['to_type'].isin(tvl['top_level_name'])
                                ,distrib_df['to_type']
                                ,''
                                )

sum_distrib_df = distrib_df[['from_name','op_deployed','to_label','to_type','program_map']].groupby(['from_name','program_map']).sum()
sum_distrib_df.reset_index(inplace=True)
display(sum_distrib_df)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_23367/3580340657.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_distrib_df = distrib_df[['from_name','op_deployed','to_label','to_type','program_map']].groupby(['from_name','program_map']).sum()


,from_name,program_map,op_deployed
0,1inch,,300000
1,Aelin,,900002
2,Alchemix,,20386
3,Alchemix,alETH/WETH: Velodrome,10000
4,Angle,,22696
...,...,...,...
93,Zipswap,,2469
94,dForce,,555913
95,dHedge,,189515
96,dHedge,DHT/OP: Velodrome,10219
